# Исследование надежности заемщиков.

Описание проекта:
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Обзор данных

Читаем таблицу и сохраняем её в переменной borrowers (анг. заёмщики)
Выводим таблицу на экран:

In [1]:
import pandas as pd

In [2]:
borrowers = pd.read_csv('/datasets/data.csv')

In [3]:
borrowers

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Выгружаем информацию о таблице:

In [4]:
borrowers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Рассмотрим входные данные. Мы емеем таблицу, в которой содержатся следующие данные:
children — количество детей в семье
days_employed — общий трудовой стаж в днях
dob_years — возраст клиента в годах
education — уровень образования клиента
education_id — идентификатор уровня образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита
В таблице 12 столбцов и 21525 строк, т.е. нам предоставлены данные о 21525 клиентах Банка, на основании которых мы будем проводить анализ.

### Шаг 2.1 Заполнение пропусков

In [5]:
borrowers.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Выясняется, что пропущенные значения присутствуют в двух столбцах: 'days_employed'(общий трудовой стаж в днях) и 'total_income'
(ежемесячный доход). При этом количество пропущенных значений в обоих столбцах одинаковое. Из информации о таблице мы видим,
что в ней всего 21525 строк, т.о. пропуски в обоих столбцах составляют (2174/21525) 10.099883% - большая доля.
Столбцы с пропусками имеют тип данных float64 - это числовой тип данных. Проверим, совпадают ли строки с пропусками в этих двух столбцах:

In [6]:
borrowers_passed = borrowers.loc[borrowers['total_income'].isna()]
borrowers_passed

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Как я и ожидала - совпадают. Т.е. информация о доходах и о стаже отсутствует у одних и тех же клиентов. Посмотрим, к какому типу занятости относятся эти клиенты. Сначала выгрузим типы занятости в таблице с пропусками (borrowers_passed), затем в общей таблице (borrowers):

In [7]:
borrowers_passed['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [8]:
borrowers['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

Получается, что информация о доходах и о стаже отсутствует у тех клиентов, у которых она объективно должна быть.
Эти клиенты не являются студентами, безработными и т.п. Значит причиной отсутствия не может быть реальное отсутствие дохода
и стажа. Посмотрим, имеются ли задолженности у этих клиентов:

In [9]:
borrowers_passed.groupby('income_type')['debt'].value_counts()

income_type      debt
госслужащий      0        140
                 1          7
компаньон        0        478
                 1         30
пенсионер        0        378
                 1         35
предприниматель  0          1
сотрудник        0       1007
                 1         98
Name: debt, dtype: int64

Если предположить, что значение 1 в столбце "dept" означает наличие задолженности, а значение 0 означает отсутствие задолженности, мы приходим к выводу, что процент должников среди клиентов с пропущенными данными не значителен, поэтому предполагать, что данные пропущены из-за потери источника дохода, не приходится. Вариант с отсутствием информации, которую предоставлял клиент тоже маловероятен, так как кредит ему всё-таки выдали, а значит данные о стаже и доходах он предоставлял.
Т.о. мы приходим к выводу, что информация отсутствует из-за технического сбоя. Лучше всего было бы уточнить этот момент у отправителя файла, возможно у него есть данные, но в нашем случае это невозможно, поэтому будем работать с тем, что есть.


Займемся пропусками в столбце 'total_income'(ежемесячный доход). Данные в этом столбце содержат количественные значения. 
Пропуски в таких переменных заполняют характерными значениями. Это значения, характеризующие состояние выборки, — набора данных, выбранных для проведения исследования. Чтобы примерно оценить типичные значения выборки, годятся среднее арифметическое или медиана. Мы будем использовать медианное значение. Сначала найдем медианное значение для каждой категории клиентов:

In [10]:
borrowers_transformed = borrowers.groupby('income_type')['total_income'].mean()
borrowers_transformed

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64

Затем пишем функцию, которая вставит медианные значения для каждой категории клиентов вместо пропусков:

In [11]:
def func(row):
    if pd.isna(row['total_income']):
        return borrowers_transformed.loc[row['income_type']]
    return row['total_income']
 
borrowers['total_income'] = borrowers.apply(func, axis=1) 

Теперь проверим, сработала ли функция - запросим количество пропусков после всех проведенных манипуляций:

In [12]:
borrowers.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

Итог: мы избавились от пропусков в столбце 'total_income', заменив их медианными значениями для каждой отдельной группы клиентов.

### Шаг 2.2 Проверка данных на аномалии и исправления.

Теперь займёмся столбцом 'days_employed'. В нём отображается трудовой стаж. Единицей измерения является день. Выгрузим минимальное и максимальное значение в столбце:

In [13]:
print(borrowers['days_employed'].min())
print(borrowers['days_employed'].max())

-18388.949900568383
401755.40047533


Первое, что бросается в глаза - это значения в столбце со знаком "-". Как количество дней может быть отрицательным? Вторая аномалия - максимальное значение (401755.40047533). Если перевести в годы, то трудовой стаж этого клиента составляет 1100 лет - такого объективно быть не может.

Начнем с первой проблемы. Вероятнее всего это технический сбой - логической причины такого явления я найти не могу. Значит надо избавиться от знака минус в наших данных:

In [14]:
borrowers['days_employed']=borrowers['days_employed'].abs()

Теперь проверим результат, снова запросив минимальное и максимальное значение:

In [15]:
print(borrowers['days_employed'].min())
print(borrowers['days_employed'].max())

24.14163324048118
401755.40047533


Вторая проблема - аномальное большие значения стажа - могла возникнуть в результате сбоя или произошла ошибка в единицах измерения. Хотя нам обозначено, что стаж указан в днях, скорее всего это не так. Мы могли бы запросить данные еще раз. уточнить возможность такой ситуации, но у нас нет такой возможности. Поэтому предлагаю решить вопрос следующим образом: расчитаем максимально возможный трудовой стаж исходя из максимального возраста клиента и заменим этим значением значения, превышающие его. Сначала находим максимальный возраст клиента в нашей таблице:

In [16]:
borrowers['dob_years'].max()

75

Самому старшему клиенту нашей таблицы 75 лет. С учетом трудового законодательства он мог начать работать в 14 лет (самое раннее), значит его максимально возможный стаж составляет 61 год или 22265 дней. Заменим этим значением все значения, его превышающие, а затем сразу проверим, запросив максимальное значение стажа:

In [17]:
borrowers.loc[borrowers['days_employed'] > 22265, 'days_employed'] = 22265
borrowers['days_employed'].max()

22265.0

Теперь, когда значения столбца адекватны, посмотрим на среднее значение стажа в различных категориях клиентов:

In [18]:
borrowers_transformed_second = borrowers.groupby('income_type')['days_employed'].mean()
borrowers_transformed_second

income_type
безработный        22265.000000
в декрете           3296.759962
госслужащий         3399.896902
компаньон           2111.524398
пенсионер          22265.000000
предприниматель      520.848083
сотрудник           2326.499216
студент              578.751554
Name: days_employed, dtype: float64

Вероятно этими значениями будет целесообразно заменить пропуски. Воспользуемся для этого ранее применяемой функцией:

In [19]:
def func_second(row):
    if pd.isna(row['days_employed']):
        return borrowers_transformed_second.loc[row['income_type']]
    return row['days_employed']
 
borrowers['days_employed'] = borrowers.apply(func_second, axis=1) 

Проверим, сработала ли функция:

In [20]:
borrowers.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Шаг 2.3. Изменение типов данных.

Теперь осталось заменить вещественный тип данных в столбцах total_income и days_employed, чтобы значения были проще для 
восприятия:

In [21]:
borrowers['total_income'] = borrowers['total_income'].astype(int)
borrowers['days_employed'] = borrowers['days_employed'].astype(int)
borrowers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Сначала поищем дубликаты в ручном режиме. Выясним, сколько всего дубликатов есть в нашей таблице:

In [22]:
borrowers.duplicated().sum()

54

Это явные строки-дубликаты. Удалим их методом drop_duplicates() с заменой индексов reset_index(drop=True)  и сразу проверим, избавились ли мы от них:

In [23]:
borrowers = borrowers.drop_duplicates().reset_index(drop=True)

In [24]:
borrowers.duplicated().sum()

0

Теперь предлагаю подгрузить уникальные значения "категориальных" столбцов, чтобы картина стала понятнее. Такими столбцами у нас являются education, family_status, gender, income_type, purpose.

In [25]:
borrowers['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [26]:
borrowers['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [27]:
borrowers['gender'].value_counts()

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

In [28]:
borrowers['income_type'].value_counts()

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [29]:
borrowers['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Очевидно, что у нас проблемы в двух столбцах: purpose и education. Скорее всего данные вводили вручную - это привело к различным выражениям в столбце purpose. 
В столбце education дубликаты появились из-за различий в регистрах написания по той же причине. Решить этот вопрос легко - приведем все значения к единому регистру:

In [30]:
borrowers['education'] = borrowers['education'].str.lower()

Проверим, что у нас получилось - снова запросим уникальные значения столбца education:

In [31]:
borrowers['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Со строками-дубликатами и дубликатами в столбце education мы разобрались. К столбцу purpose мы вернемся позднее (в 7-ом задании второго шага).

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим 2 новых датафрейма. 
В первом каждому уникальному значению из education соответствует уникальное значение education_id:

In [32]:
borrowers_education = borrowers[['education','education_id']]
borrowers_education

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1
...,...,...
21466,среднее,1
21467,среднее,1
21468,среднее,1
21469,среднее,1


Во втором каждому уникальному значению из family_status соответствует уникальное значение family_status_id:

In [33]:
borrowers_family_status = borrowers[['family_status','family_status_id']]
borrowers_family_status

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1
...,...,...
21466,гражданский брак,1
21467,женат / замужем,0
21468,гражданский брак,1
21469,женат / замужем,0


Эти новые датафреймы - наши "словари". В них сейчас много дубликатов, которые надо удалить:

In [34]:
borrowers_education = borrowers_education.drop_duplicates().reset_index(drop=True)
borrowers_education

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [35]:
borrowers_family_status = borrowers_family_status.drop_duplicates().reset_index(drop=True)
borrowers_family_status

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Теперь можно удалить из исходного датафрейма столбцы education и family_status, оставив столбцы education_id и family_status_id, по которым в словарях мы будем искать значения. Сначала удалим столбец education:

In [36]:
borrowers.drop('education', inplace=True, axis=1)

Затем удалим столбец family_status и выгрузим таблицу для проверки:

In [37]:
borrowers.drop('family_status', inplace=True, axis=1)
borrowers

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,22265,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21466,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем
21467,0,22265,67,1,0,F,пенсионер,0,155999,сделка с автомобилем
21468,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость
21469,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля


Теперь исходная таблица стала легче для восприятия и проще в работе: по значению столбцов education_id и family_status_id мы в новых таблицах-словарях легко находим описание указанных статусов.

### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создаём столбец total_income_category с категориями:
0–30000 — 'E';
30001–50000 — 'D';
50001–200000 — 'C';
200001–1000000 — 'B';
1000001 и выше — 'A'.
Для этого пишем функцию с правилами классификации клиентов. Учтем в функции возможность наличия значений, выпадающих из наших категорий - будем откладывать их в список error_list с помощью конструкции try-except. Назовём её age-group:

In [38]:
error_list   = []
def age_group(total_income):
    try:
        if 0<=total_income<=30000:
            return 'E'
        if 30001<=total_income<=50000:
            return 'D'
        if 50001<=total_income<=200000:
            return 'C'
        if 200001<=total_income<=1000000:
            return 'B'
        if total_income>=1000001:
            return 'A'
    except:
        error_list.append(total_income)
        

Теперь создаем новый столбец total_income_category с категориями:

In [39]:
borrowers['total_income_category'] = borrowers['total_income'].apply(age_group)

Прверим, добавился ли новый столбец и сработала ли функция:

In [40]:
borrowers

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,22265,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
...,...,...,...,...,...,...,...,...,...,...,...
21466,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем,B
21467,0,22265,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C
21468,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость,C
21469,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B


Проверим так же, пополнился ли наш список ошибок:

In [41]:
error_list

[]

Не пополнился, значит других значений в столбце нет.

### Шаг 2.7. Категоризация целей кредита.

Вернемся к столбцу purpose (цель кредита). Как мы помним, в нем очень много дубликатов, которые возникли скорее всего из-за ручного ввода данных. Еще раз выгрузим уникальные значения этого столбца и убедимся в этом:

In [42]:
borrowers['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Напишем функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:
'операции с автомобилем',
'операции с недвижимостью',
'проведение свадьбы',
'получение образования'.

In [43]:
def purpose_group(purpose):
    if 'свадь' in purpose:
        return 'проведение свадьбы'
    if 'автом' in purpose:
        return 'операции с автомобилем'
    if 'образ' in purpose:
        return 'получение образования'
    if 'недвиж' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'

Теперь сформируем новый столбец, в который войдут эти категории и выведем на экран датафрейм для проверки:

In [44]:
borrowers['purpose_category'] = borrowers['purpose'].apply(purpose_group)
borrowers

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,22265,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21467,0,22265,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21468,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21469,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем


### Ответы на вопросы.

##### Вопрос 1:

Выясним, есть ли зависимость между количеством детей и возвратом кредита в срок. Для этого выгрузим количество неплательщиков в каждой группе родителей. Примем за истину то, что значение 1 в столбце debt означает наличие задолженности, а значение 0 -её отсутствие. Воспользуемся методом pivot_table(), чтобы данные были удобнее для прочтения:

In [45]:
borrowers_children = borrowers.pivot_table(index='children', columns='debt', values='income_type', aggfunc='count')
borrowers_children

debt,0,1
children,,
-1,46.0,1.0
0,13044.0,1063.0
1,4365.0,444.0
2,1858.0,194.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN
20,68.0,8.0


Мы видим, что всплыла проблема: -1 ребенок у человека быть не может. Стоило бы уточнить информацию у отправителя данных, но у нас такой возможности нет, так что будем предполагать, что это технический сбой и имелось ввиду количество детей 1. Для того, чтобы данные выгружались корректно, исправим данные в столбце:

In [46]:
borrowers['children']=borrowers['children'].abs()
borrowers_children = borrowers.pivot_table(index='children', columns='debt', values='income_type', aggfunc='count')
borrowers_children

debt,0,1
children,,
0,13044.0,1063.0
1,4411.0,445.0
2,1858.0,194.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN
20,68.0,8.0


Посмотрим, какова доля должников среди клиентов в зависимости от того, сколько у них детей - введем дополнительный столбец, отражающий эту информацию:

In [47]:
borrowers_children['share_debtors'] = borrowers_children[1] / borrowers_children[0] * 100
borrowers_children

debt,0,1,share_debtors
children,,,
0,13044.0,1063.0,8.149341
1,4411.0,445.0,10.088415
2,1858.0,194.0,10.441335
3,303.0,27.0,8.910891
4,37.0,4.0,10.810811
5,9.0,NaN,NaN
20,68.0,8.0,11.764706


##### Вывод 1:

Что мы видим:
Неплательщиков среди клиентов с 1 ребенком - 10,0%
Неплательщиков среди клиентов с 2 детьми  - 10,4%
Неплательщиков среди клиентов с 3 детьми - 8,9% - этих клиентов крайне мало, рассматривать данные по неплательщикам в этой категории не стоит. Такая же ситуация с 4 и более детьми. Связано это скорее всего с тем, что таких семей не так много. 
Нужно учитывать тот факт, что отдельно от прочих условий рассматривать причиной задолженности только количество детей впринципе нецелесообразно, однако при такой категоричности вопроса можно сделать вывод, что семьи с одним ребенком справляются с платежами лучше, чем прочие.

0.9.0.3 Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок? Для этого выгрузим количество неплательщиков в каждой группе семейного положения. Опять же за истину примем то, что значение 1 в столбце debt означает наличие задолженности, а значение 0 -её отсутствие. Нам нужно будет обратиться к наше "таблице-словарю", так как ранее мы удалили столбец family_status.

In [48]:
borrowers_family_status_id = borrowers.pivot_table(index='family_status_id', columns='debt', values='income_type', aggfunc='count')
borrowers_family_status_id

debt,0,1
family_status_id,,
0,11413,931
1,3775,388
2,896,63
3,1110,85
4,2536,274


In [49]:
borrowers_family_status

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Посмотрим, какова доля должников среди клиентов в зависимости от семейного положения - введем дополнительный столбец, отражающий эту информацию:

In [50]:
borrowers_family_status_id['share_debtors'] = borrowers_family_status_id[1] / borrowers_family_status_id[0] * 100
borrowers_family_status_id

debt,0,1,share_debtors
family_status_id,,,
0,11413,931,8.157364
1,3775,388,10.278146
2,896,63,7.031250
3,1110,85,7.657658
4,2536,274,10.804416


0.9.0.4 Вывод 2:

Мы видим, что:
среди женатых неплательщиков - 8,1%
среди живущих в гражданском браке - 10,3%
среди вдовцов - 7,0% их крайне мало в нашей таблице - ориентироваться на эти данные мы не будем.
среди людей в разводе - 7,7%
среди холостяков - 10,8%
Казалось бы, что среди холостяков и живущих в гражданском браке неплательщиков больше, но я настаиваю на том, что брать один показатель без учета прочих вводных данных - решение неконструктивное.

0.9.0.5 Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок? Для этого выгрузим количество неплательщиков в каждой группе дохода. Для этого прибегнем к созданному ранее столбцу с категориями клиентов по доходам:

In [51]:
borrowers_total_income_category = borrowers.pivot_table(index='total_income_category', columns='debt', values='income_type', aggfunc='count')
borrowers_total_income_category

debt,0,1
total_income_category,,
A,23,2
B,5159,386
C,14199,1330
D,329,21
E,20,2


Посмотрим, какова доля должников среди клиентов в зависимости от их дохода - введем дополнительный столбец, отражающий эту информацию:

In [52]:
borrowers_total_income_category['share_debtors'] = borrowers_total_income_category[1] / borrowers_total_income_category[0] * 100
borrowers_total_income_category

debt,0,1,share_debtors
total_income_category,,,
A,23,2,8.695652
B,5159,386,7.482070
C,14199,1330,9.366857
D,329,21,6.382979
E,20,2,10.000000


0.9.0.6 Вывод 3:

Мы помним, что категории распределялись следующим образом:  0–30000 — 'E'; 30001–50000 — 'D'; 50001–200000 — 'C'; 200001–1000000 — 'B'; 1000001 и выше — 'A'
Среди клиентов категории 'A' - 8,7% неплательщиков
Среди клиентов категории 'B' - 7,5% неплательщиков
Среди клиентов категории 'C' - 9,4% неплательщиков
Среди клиентов категории 'D' - 6,4% неплательщиков
Среди клиентов категории 'E' - 10,0% неплательщиков
Прямой зависимости мы не видим, приблизительно равна доля неплательщиков и среди состоятельных людей, и среди людей со средним доходом, и среди тех, кто зарабатывает менее 30 000. Но опять же ориентироваться только на этот показатель будет неверно только потому, что клиентов категории 'A' и 'E' очень мало по сравнению с клиентами категории 'C'.

0.9.0.7 Вопрос 4:

Как разные цели кредита влияют на его возврат в срок? Мы разбивали по категориям цели кредита. Снова воспользуемся pivot_table():

In [53]:
borrowers_purpose_category = borrowers.pivot_table(index='purpose_category', columns='debt', values='income_type', aggfunc='count')
borrowers_purpose_category

debt,0,1
purpose_category,,
операции с автомобилем,3905,403
операции с недвижимостью,10032,782
получение образования,3644,370
проведение свадьбы,2149,186


Посмотрим, какова доля должников среди клиентов в зависимости от цели кредита - введем дополнительный столбец, отражающий эту информацию:

In [54]:
borrowers_purpose_category['share_debtors'] = borrowers_purpose_category[1] / borrowers_purpose_category[0] * 100
borrowers_purpose_category

debt,0,1,share_debtors
purpose_category,,,
операции с автомобилем,3905,403,10.320102
операции с недвижимостью,10032,782,7.795056
получение образования,3644,370,10.153677
проведение свадьбы,2149,186,8.655188


0.9.0.8 Вывод 4:

Из выгрузки мы видим следующее: 
неплательщиков среди тех, кто брал кредит на операции с автомобилем - 10,3%
неплательщиков среди тех, кто брал кредит на операции с недвижимостью - 7,8%
неплательщиков среди тех, кто брал кредит на получение образования - 10,1%
неплательщиков среди тех, кто брал кредит на проведение свадьбы - 8,7%
Можно сделать вывод, что больше всего неплательщиков среди тех, кто преобретал автомобиль, и меньше всего среди тех, кто покупал недвижимость. Однако снова акцентирую внимание на том, что по одному параметру оценивать плательщика нецелесообразно, так как скорее всего к клиенту, желающему взять в кредит недвижимость требования скорее всего выше, так как суммы таких кредитов больше.

## Общий вывод:

Казалось бы мы нашли идеальный портрет заемщика: семейный человек с одним ребенком, доходом 200001–1000000 рублей, берущий кредит на решение вопросов с недвижимостью. Однако нельзя учитывать только эти характеристики. Во-первых выборка крайне неравномерная - количетсво клиентов в каждой категории этих выборок очень сильно разнится. Во-вторых, требования к клиентам, берущим кредиты на различные нужды, тоже разнятся (нужен разный комплект документов, наличие/отсутствие залога, поручителя и т.п.), неизвестно, насколько стабилен доход (в какой сфере работает клиент, уверены ли мы, что его доход будет прежним через месяц, квартал, год).